<a href="https://colab.research.google.com/github/shruthi-sh/Capstone--CLS/blob/main/Pegasus_lstm_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pegasus summerizer + Translation(LSTM) Testing
Bleu score - 20.701

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# importing required modules 
from zipfile import ZipFile 
  
import gzip
import shutil
with gzip.open('/content/drive/My Drive/cc.en.300.vec.gz', 'rb') as f_in:
    with open('file2.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
import numpy as np 
embeddings_index_en = {}
f = open('file2.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_en[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_en))

Found 2000000 word vectors.


In [ ]:
x=list(embeddings_index_en.keys())
y=list(embeddings_index_en.values())
print(x[222],y[222])

does [ 0.0255  0.0185 -0.0066  0.0644 -0.0781 -0.0613  0.0004 -0.0637  0.079
 -0.0612 -0.0619  0.0334  0.0188 -0.0439 -0.043   0.0515  0.0424 -0.0408
  0.0275  0.0901 -0.0477  0.0116  0.0271  0.0672 -0.0811  0.022  -0.0119
  0.0189  0.0361  0.041  -0.0258  0.068   0.0385 -0.0511 -0.0067 -0.0296
 -0.1037  0.0348  0.0652 -0.0658 -0.0545 -0.0514  0.0359 -0.0074  0.0444
  0.1143 -0.0251 -0.0618  0.0431 -0.0463 -0.0346 -0.0648  0.0136  0.0789
 -0.0436  0.0261 -0.0259  0.1028 -0.0891  0.0186  0.005  -0.0037 -0.0629
  0.0124 -0.0105  0.0005 -0.0504 -0.0073 -0.027   0.0048 -0.0661 -0.0671
  0.039  -0.0289  0.0262 -0.0224  0.1326  0.0272 -0.1178 -0.0884 -0.0823
 -0.0069 -0.031   0.1139 -0.1622  0.0021  0.0038 -0.0713  0.0319 -0.0153
  0.03   -0.039   0.1076 -0.0263  0.1331 -0.0985 -0.1275  0.0162 -0.0126
  0.0351  0.0355  0.0494  0.0036 -0.0334 -0.0704  0.1096  0.0655  0.0727
 -0.0346  0.0176  0.1153  0.0174 -0.01   -0.1314 -0.0735 -0.0679 -0.0904
 -0.0513  0.0007 -0.078   0.051   0.0872  0.077

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/training_final_1.csv')
df.head()


,Unnamed: 0,Document,Summary,Kannada_summary
0,0,It's official: U.S. President Barack Obama wan...,Syrian official: Obama climbed to the top of t...,"ಸಿರಿಯನ್ ಅಧಿಕಾರಿ: ಒಬಾಮಾ ಮರದ ಮೇಲಕ್ಕೆ ಏರಿದರು, ""ಹೇ..."
1,1,(CNN) -- Usain Bolt rounded off the world cham...,Usain Bolt wins third gold of world championsh...,ಉಸೇನ್ ಬೋಲ್ಟ್ ವಿಶ್ವ ಚಾಂಪಿಯನ್‌ಶಿಪ್‌ನ ಮೂರನೇ ಚಿನ್ನ...
2,2,"Kansas City, Missouri (CNN) -- The General Ser...",The employee in agency's Kansas City office is...,ಏಜೆನ್ಸಿಯ ಕಾನ್ಸಾಸ್ ಸಿಟಿ ಕಚೇರಿಯಲ್ಲಿ ಉದ್ಯೋಗಿ ನೂರಾ...
3,3,Los Angeles (CNN) -- A medical doctor in Vanco...,NEW: A Canadian doctor says she was part of a ...,ಹೊಸ: ಕೆನಡಾದ ವೈದ್ಯರೊಬ್ಬರು 2010 ರಲ್ಲಿ ಹ್ಯಾರಿ ಬುರ...
4,4,(CNN) -- Police arrested another teen Thursday...,Another arrest made in gang rape outside Calif...,ಕ್ಯಾಲಿಫೋರ್ನಿಯಾ ಶಾಲೆಯ ಹೊರಗೆ ಸಾಮೂಹಿಕ ಅತ್ಯಾಚಾರದಲ್...


In [ ]:

df=df.drop(columns="Document")
df=df.drop(columns="Unnamed: 0")
len(df)

90232

In [ ]:
def cleanerEng(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'[^a-z0-9]+',' ',x)
  if len(x) > 150:
    x = x[:150]
  return x

def cleanerkan(x):
  x = str(x)
  x = re.sub(r'[-.।|,?;:<>&$₹]+',' ',x)
  if len(x) > 150:
    x = x[:150]
  return x

In [ ]:
import re
df.iloc[:,0] = df['Summary'].apply(func=cleanerEng)
df.iloc[:,1] = df['Kannada_summary'].apply(func= cleanerkan)
df.iloc[:,0] = df['Summary'].apply(func= lambda x : (str(x).split()))
df.iloc[:,1] = df['Kannada_summary'].apply(func= lambda x : (str(x).split()))

In [ ]:
df.head()

,Summary,Kannada_summary
0,"[syrian, official, obama, climbed, to, the, to...","[ಸಿರಿಯನ್, ಅಧಿಕಾರಿ, ಒಬಾಮಾ, ಮರದ, ಮೇಲಕ್ಕೆ, ಏರಿದರು..."
1,"[usain, bolt, wins, third, gold, of, world, ch...","[ಉಸೇನ್, ಬೋಲ್ಟ್, ವಿಶ್ವ, ಚಾಂಪಿಯನ್‌ಶಿಪ್‌ನ, ಮೂರನೇ,..."
2,"[the, employee, in, agency, s, kansas, city, o...","[ಏಜೆನ್ಸಿಯ, ಕಾನ್ಸಾಸ್, ಸಿಟಿ, ಕಚೇರಿಯಲ್ಲಿ, ಉದ್ಯೋಗಿ..."
3,"[new, a, canadian, doctor, says, she, was, par...","[ಹೊಸ, ಕೆನಡಾದ, ವೈದ್ಯರೊಬ್ಬರು, 2010, ರಲ್ಲಿ, ಹ್ಯಾರ..."
4,"[another, arrest, made, in, gang, rape, outsid...","[ಕ್ಯಾಲಿಫೋರ್ನಿಯಾ, ಶಾಲೆಯ, ಹೊರಗೆ, ಸಾಮೂಹಿಕ, ಅತ್ಯಾಚ..."


In [ ]:
def addTokens(x,start=False):
  x.append('<END>')
  if start:
    x.insert(0,'<START>')
  return list(x)

In [ ]:


df.iloc[:,0] = df['Summary'].apply(func= addTokens,start=False)
df.iloc[:,1] = df['Kannada_summary'].apply(func= addTokens,start=True)

In [ ]:
df.iloc[79,1]

['<START>',
 'ಹೊಸ',
 'ಕ್ರೇನ್',
 'ಆಪರೇಟರ್',
 'ಪರ',
 'ವಕೀಲರು',
 'ತಮ್ಮ',
 'ಕ್ಲೈಂಟ್',
 'ಅನ್ನು',
 '"ಬಲಿಪಶುವನ್ನಾಗಿ',
 'ಮಾಡಲಾಗುತ್ತಿದೆ"',
 'ಎಂದು',
 'ಹೇಳುತ್ತಾರೆ',
 'ಸೀನ್',
 'ಬೆನ್\u200cಶಾಪ್\u200cಗೆ',
 'ಭಾನುವಾರ',
 'ಜಾಮೀನು',
 'ನಿರಾಕರಿಸಲಾಗಿದೆ',
 'ಕ್ರೇನ',
 '<END>']

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random

In [ ]:
data = df.values

In [ ]:
data[:,1].shape

(90232,)

In [ ]:
class vocab:

  def __init__(self,data,token=True):
    self.data = data
    if token:
      self.word2idx = {'<START>':1, '<END>':2, '<PAD>':0}
      self.idx2word = {1:'<START>', 2:'<END>', 0:'<PAD>'}
      self.idx = 2

    else:
      self.word2idx = {'<PAD>':0, '<END>':1}
      self.idx2word = {0:'<PAD>', 1:'<END>'}
      self.idx = 1

    self.x = []
    self.create()
    self.vocab_size = self.idx + 1

  def create(self):
    max_len = 0;
    for sentence in  self.data:
      max_len = max(max_len, len(sentence))
      for word in sentence:
        if self.word2idx.get(word) is None:
          self.idx += 1
          self.word2idx[word] = self.idx
          self.idx2word[self.idx] = word
    
    for sentence in self.data:
      sent = []
      for word in sentence:
        sent.append(self.word2idx[word])
      
      for i in range(len(sentence),max_len+1):
        sent.append(0)
      
      self.x.append(torch.Tensor(sent))


In [ ]:
English_vocab = vocab(data[:,0],token=False)
kan_vocab = vocab(data[:,1],token=True)

In [ ]:
print(kan_vocab)

In [ ]:
for idx in English_vocab.x[0]:
  print(English_vocab.idx2word[int(idx)],end=' ')
  print('('+str(int(idx))+')',end=' ')


syrian (2) official (3) obama (4) climbed (5) to (6) the (7) top (8) of (9) the (7) tree (10) doesn (11) t (12) know (13) how (14) to (6) get (15) down (16) obama (4) sends (17) a (18) letter (19) to (6) the (7) heads (20) of (9) the (7) house (21) and (22) senate (23) obama (4) to (6) s (24) <END> (1) <PAD> (0) <PAD> (0) <PAD> (0) <PAD> (0) <PAD> (0) <PAD> (0) 

In [ ]:
for idx in kan_vocab.x[3]:
  print(kan_vocab.idx2word[int(idx)],end=' ')
  

<START> ಹೊಸ ಕೆನಡಾದ ವೈದ್ಯರೊಬ್ಬರು 2010 ರಲ್ಲಿ ಹ್ಯಾರಿ ಬುರ್ಖಾರ್ಟ್ ಅವರನ್ನು ಪರೀಕ್ಷಿಸುವ ತಂಡದ ಭಾಗವಾಗಿದ್ದರು ಎಂದು ಹೇಳಿದ್ದಾರೆ ಹೊಸ ರೋಗನಿರ್ಣಯ "ಸ್ವಲೀನತೆ ತೀವ್ರ ಆತಂಕ ನಂತ <END> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 

In [ ]:
class parallelData(Dataset):

  def __init__(self):
    self.x = English_vocab.x
    self.y = kan_vocab.x

  def __getitem__(self,i):
    return self.x[i], self.y[i]
  
  def __len__(self):
    return len(self.x)

In [ ]:
dataset = parallelData()

In [ ]:
a = dataset[0][0].shape[0]
b = dataset[0][1].shape[0]
for i in range(len(dataset)):
  if a != dataset[i][0].shape[0] or b != dataset[i][1].shape[0]:
    print(a,dataset[i][0].shape[0],b,dataset[i][1].shape[0])

In [ ]:

kan_vocab.x[90].shape

torch.Size([33])

In [ ]:
torch.cuda.device_count()

1

In [ ]:
print(English_vocab.word2idx.items())

dict_items([('<PAD>', 0), ('<END>', 1), ('syrian', 2), ('official', 3), ('obama', 4), ('climbed', 5), ('to', 6), ('the', 7), ('top', 8), ('of', 9), ('tree', 10), ('doesn', 11), ('t', 12), ('know', 13), ('how', 14), ('get', 15), ('down', 16), ('sends', 17), ('a', 18), ('letter', 19), ('heads', 20), ('house', 21), ('and', 22), ('senate', 23), ('s', 24), ('usain', 25), ('bolt', 26), ('wins', 27), ('third', 28), ('gold', 29), ('world', 30), ('championship', 31), ('anchors', 32), ('jamaica', 33), ('4x100m', 34), ('relay', 35), ('victory', 36), ('eighth', 37), ('at', 38), ('championships', 39), ('for', 40), ('double', 41), ('up', 42), ('i', 43), ('employee', 44), ('in', 45), ('agency', 46), ('kansas', 47), ('city', 48), ('office', 49), ('is', 50), ('among', 51), ('hundreds', 52), ('virtual', 53), ('workers', 54), ('travel', 55), ('from', 56), ('mainland', 57), ('u', 58), ('last', 59), ('year', 60), ('cost', 61), ('mor', 62), ('new', 63), ('canadian', 64), ('doctor', 65), ('says', 66), ('she'

In [ ]:
English_vocab.vocab_size

69134

In [ ]:
kan_vocab.vocab_size

162962

In [ ]:
print(embeddings_index_en.get('top'))

[-1.252e-01  1.788e-01 -1.641e-01  2.106e-01 -1.940e-02  5.750e-02
  2.360e-01 -9.400e-03 -2.581e-01  1.780e-02 -1.025e-01  9.900e-02
  4.460e-02  1.813e-01 -7.210e-02 -5.950e-02  7.220e-02 -1.500e-03
 -6.790e-02 -9.350e-02 -1.330e-02 -3.950e-02 -8.320e-02 -5.450e-02
  1.268e-01  3.600e-03  1.362e-01  1.040e-01  4.210e-02 -1.000e-02
  7.410e-02  2.470e-02 -9.000e-03 -1.321e-01 -1.077e-01 -1.095e-01
 -1.985e-01  4.920e-02 -1.116e-01 -7.920e-02 -1.854e-01  1.080e-02
 -4.090e-02 -4.210e-02 -1.286e-01 -1.640e-02  2.060e-02 -3.390e-02
 -3.720e-02  2.000e-02 -9.520e-02  6.160e-02  2.933e-01 -1.769e-01
 -6.380e-02 -1.816e-01 -4.890e-02  2.580e-02 -7.100e-02 -2.430e-01
  1.556e-01 -2.920e-02 -1.686e-01 -1.323e-01  1.242e-01 -1.303e-01
 -1.030e-01 -1.250e-01 -1.059e-01 -5.420e-02  1.968e-01  1.759e-01
  3.450e-02  1.220e-01  5.800e-02  2.400e-03 -1.920e-02 -1.278e-01
  1.409e-01 -2.380e-02  7.990e-02 -1.580e-02 -6.260e-02 -4.990e-02
 -5.780e-02  1.969e-01 -6.730e-02  5.600e-03 -3.600e-02  1.076

In [ ]:

#creating input embedding matrix
embedding_dim = 300
embedding_matrix_en = np.zeros((English_vocab.vocab_size, embedding_dim))
for word, i in English_vocab.word2idx.items() :
    embedding_vector = embeddings_index_en.get(word)
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_en[i] = embedding_vector
        


In [ ]:
class encoder(nn.Module):

  def __init__(self, input_size, embedding_size, hidden_size, layers, bidirectional):
    '''
    input_size = size of vocab
    embedding_size = embedding dim
    hidden_size = hidden state size
    layer = num of layers of lstms
    '''
    super().__init__()
    self.embed = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size) # output size = (*,embedding_size)
    self.embed.weight.data.copy_=(embedding_matrix_en)
    self.lstm = nn.LSTM(input_size=embedding_size, hidden_size= hidden_size, num_layers=layers, batch_first = True, bidirectional = bidirectional)
    self.bidirectional = bidirectional
    #in order to convert bidirectional hidden state to unidirectional if LSTM is bidirectional 
    self.fc_hidden = nn.Linear(hidden_size*2, hidden_size)
    self.fc_cell = nn.Linear(hidden_size*2, hidden_size)
  def forward(self,x):
    '''
    x shape = [batch_size, sentence]
    one complete sentence represents a "sequence"
    '''
    x = self.embed(x) # shape [batch_size,  sentence, embed_size]
    output, (hidden_state, cell_state) = self.lstm(x) #shape [batch_size, seq_len, num_directions(2)*hidden_size]

    if self.bidirectional:  #since we have 2 directions so add(concat) hidden of both directions into one
      hidden = torch.cat((hidden_state[0:1], hidden_state[1:2]), dim=2)
      cell = torch.cat((cell_state[0:1], cell_state[1:2]), dim = 2) #output [1(layer), batch, hidden_size*2]
      hidden_state = self.fc_hidden(hidden)
      cell_state = self.fc_cell(cell)

    # print(output.shape, x.shape)
    #output shape = [batch_size, seq_len, 2*hidden_size] 
    #hidden shape =[1(layers), batch_size, hidden_size]
    return output, hidden_state, cell_state


In [ ]:
"""
class decoder(nn.Module):

  def __init__(self,input_size, embedding_size, hidden_size, layers):
    '''
    same configuration as encoder
    here input_size = size of hindi vocab
    '''
    super().__init__()
    self.embed = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size) # output size = (*,embedding_size)
    self.embed.weight.data.copy_=(embedding_matrix_kn)
    self.lstm = nn.LSTM(input_size=embedding_size, hidden_size= hidden_size, num_layers=layers, batch_first = True)
    self.fc = nn.Linear(in_features=hidden_size, out_features=input_size) #since output would be prob distribution among hindi vocab therefore out_feature=input_size

  def forward(self,x,hidden_state, cell_state):
    '''
    to have control over output we have to take sentence as word by word
    therefore seq_len would be 1 as input is  one word not the whole sentence
    x = [batch_size] ->required-> [batch_size, 1] (1 is seq_len)
    '''
    # print(x.shape)
    x = x.reshape(-1,1) # shape [batch, 1]
    # print(x.shape)
    x = self.embed(x) # shape [batch, 1, embed_dim]

    output, (hidden_state, cell_state) = self.lstm(x, (hidden_state, cell_state)) # shape output=>[batch, 1, hidden_size], hidden=>[layers, batch, hidden_size]
    output = self.fc(output) # shape [batch, 1, hindi_vocab_size]
    
    #just for removing extra dim
    output = output.squeeze(dim=1) #shape [batch, hindi_vocab_size]

    return output, hidden_state, cell_state
"""

"\nclass decoder(nn.Module):\n\n  def __init__(self,input_size, embedding_size, hidden_size, layers):\n    '''\n    same configuration as encoder\n    here input_size = size of hindi vocab\n    '''\n    super().__init__()\n    self.embed = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size) # output size = (*,embedding_size)\n    self.embed.weight.data.copy_=(embedding_matrix_kn)\n    self.lstm = nn.LSTM(input_size=embedding_size, hidden_size= hidden_size, num_layers=layers, batch_first = True)\n    self.fc = nn.Linear(in_features=hidden_size, out_features=input_size) #since output would be prob distribution among hindi vocab therefore out_feature=input_size\n\n  def forward(self,x,hidden_state, cell_state):\n    '''\n    to have control over output we have to take sentence as word by word\n    therefore seq_len would be 1 as input is  one word not the whole sentence\n    x = [batch_size] ->required-> [batch_size, 1] (1 is seq_len)\n    '''\n    # print(x.shape)\n    

In [ ]:
class AttnDecoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, layers):
    super().__init__()

    self.embed = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size) # output size = (*,embedding_size)
    self.lstm = nn.LSTM(input_size=hidden_size*2 + embedding_size, hidden_size= hidden_size, num_layers=layers, batch_first = True)
    self.fc = nn.Linear(in_features=hidden_size, out_features=input_size) #since output would be prob distribution among hindi vocab therefore out_feature=input_size

    #encoder_states from encoder => [batch, seq_len(35), 2*hidden_size]
    #prev decoder hidden_state => [batch, layers(1)*directions(2), hidden_size] =>need to be in => [batch, seq(35), hidden_size]
    #therefore input of energy will be along hidden_size ie input = hidden_size*2
    #Applies a linear transformation to the incoming data: y = xA^T + b
    self.energy =  nn.Linear(hidden_size*3, 1) #out [batch, seq_len, 1] (2 hidden state from bidirectional encoder and 1 from prev decoder hidden state => 1+2= 3 hidden states as input)
    self.softmax = nn.Softmax(dim=1)# doing softmax for each word ie (dim=1)

  
  def forward(self, x, hidden_state, cell_state, encoder_states):
    # print(encoder_states.shape)
    seq_len = encoder_states.shape[1]
    batch_size = encoder_states.shape[0]
    hidden_size = encoder_states.shape[2]

    h_new = hidden_state.repeat(seq_len, 1, 1) #shape [seq_len*1, batch, hidden_size*2(bidirectional)] it will repeat dim=0 seq length times
    #by doing .repeat operation we can concat hidden state with all timestamps of encoder_states
    # print(h_new.shape, encoder_states.shape, hidden_state.shape)
    h_new = h_new.permute(1,0,2) #[batch, seq_len, hidden_size*2]
    energy = self.energy(torch.cat((h_new, encoder_states), dim=2))#input [batch, seq_len(35), hidden_size*3]  out = [batch, seq_len(35), 1]
    att_weights = self.softmax(energy)
    att_weights = att_weights.permute(0,2,1) # [batch, 1, seq_len]

    
    context = torch.bmm(att_weights, encoder_states) #[batch, 1, hidden_size*2]

    x = x.reshape(-1,1) # shape [batch, 1]
    x = self.embed(x) # shape [batch, 1, embed_dim]

    input_new = torch.cat((context, x), dim=2) #[batch, 1, hidden_size*2 +embed_dim]

    output, (hidden_state, cell_state) = self.lstm(input_new, (hidden_state, cell_state)) # shape output=>[batch, 1, hidden_size], hidden=>[layers, batch, hidden_size]
    output = self.fc(output) # shape [batch, 1, hindi_vocab_size]

    output = output.squeeze(dim=1) #shape [batch, hindi_vocab_size]
    del h_new
    del context
    del input_new
    return output, hidden_state, cell_state

In [ ]:
class Attnseq2seq(nn.Module):
  def __init__(self, encoder, att_decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = att_decoder


  def forward(self, input, target, teaching_force=0.5):
    '''
    input = batch of english sentences[batch, sentece(padded)]
    target = batch of hindi sentences [batch, sentence(padded)] 
    '''
    batch_size = input.shape[0]
    seq_len = target.shape[1]
    kan_vocab_size = kan_vocab.vocab_size

    output = torch.zeros((seq_len, batch_size, kan_vocab_size)).to(device)

    encoder_states, hidden, cell = self.encoder(input)
    target = target.permute(1,0) # shape [seq, batch]
    x = target[0] # <START> token

    for i in range(1, seq_len):
      out, hidden, cell = self.decoder(x, hidden, cell, encoder_states) #out shape = [batch, vocab_size]
      output[i] = out
      decoder_guess = out.argmax(1)# taking the word with max value(confidence)  shape = [batch of words]
      if random.random() < teaching_force:
        x = target[i]
      else:
        x =  decoder_guess
    
    return output  #shape[seq_len, batch_size, vocab_size]


In [ ]:
##training
epochs = 10
learning_rate = 0.001
batch_size = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_dim = 300
embedding_size= 300
hidden_size = 256
layers = 1
bidirection = True


In [ ]:
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
it = iter(loader)
x,y = next(it)
print(x.shape,y.shape)
len(dataset)

torch.Size([100, 39]) torch.Size([100, 33])


90232

In [ ]:
ENC = encoder(English_vocab.vocab_size, embedding_size, hidden_size, layers, bidirection).to(device)
# DE = decoder(Hindi_vocab.vocab_size, embedding_size, hidden_size, layers).to(device)
DE = AttnDecoder(kan_vocab.vocab_size, embedding_size, hidden_size, layers).to(device)
# model = seq2seq(ENC,DE).to(device)
model = Attnseq2seq(ENC,DE).to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [ ]:
#load weights
model.load_state_dict(torch.load('/content/drive/My Drive/0.217_Copy of model_lstmFT_2.pt'))


<All keys matched successfully>

In [ ]:
def prediction(x):
    print('Summary:',end="")
    for idx in x:
      if idx == 0:
        break
      print(English_vocab.idx2word[int(idx)],end=' ')
    
    print()

    x = x.long().reshape(1,-1).to(device)
    ans = translate(x)
    res = []
    for id in ans:
      res.append(kan_vocab.idx2word[id])
    
    return res

In [ ]:
def translate(input):
      #input = batch of english sentences[batch, sentece(padded)]
      with torch.no_grad():
        guess = []
        encoder_states, hidden, cell = model.encoder(input)
        # x = torch.ones((1)).float().to(device) # <START> token
        x = torch.ones((1)).long().to(device)
        while True:
          out, hidden, cell = model.decoder(x, hidden, cell, encoder_states) #out shape = [batch, vocab_size]
          x = out.argmax(1)# taking the word with max value(confidence)  shape = [batch of words]
          guess.append(int(x[0].detach().cpu()))

          if x == 2:
            break

      return guess

In [ ]:
def get(sent):
  # sentence = sentence.lower()
  # sent = sentence.split()
  # sent.append('<END>')
  # print(sent)

  toks = []
  for word in sent:
    if English_vocab.word2idx.get(word) is None:
      toks.append(English_vocab.word2idx['the'])
    else:
      toks.append(English_vocab.word2idx[word])
  # print(toks)
  sent = torch.tensor(toks).float()
  res = prediction(sent)
  # print(res)
  return res

In [ ]:
tdf = pd.read_csv('/content/drive/My Drive/ptest_final.csv')
#tdf.drop(columns=['source'],inplace=True)
tdf.head()

,Unnamed: 0,Unnamed: 0.1,Document,Summary,Kannada_summary,PSummary
0,0,0,"(CNN)James Best, best known for his portrayal ...","James Best, who played the sheriff on ""The Duk...","""ದಿ ಡ್ಯೂಕ್ಸ್ ಆಫ್ ಹಜಾರ್ಡ್"" ನಲ್ಲಿ ಶೆರಿಫ್ ಪಾತ್ರವನ...","""I laughed and learned more from Jimmie in one..."
1,1,1,(CNN)The attorney for a suburban New York card...,A lawyer for Dr. Anthony Moschetto says the ch...,ಡಾ. ಆಂಥೋನಿ ಮೊಸ್ಚೆಟ್ಟೊ ಅವರ ವಕೀಲರು ಅವರ ವಿರುದ್ಧದ ...,Dr. Anthony Moschetto is charged in what autho...
2,2,2,(CNN)President Barack Obama took part in a rou...,"""No challenge poses more of a public threat th...","""ಹವಾಮಾನ ಬದಲಾವಣೆಗಿಂತ ಯಾವುದೇ ಸವಾಲು ಸಾರ್ವಜನಿಕ ಬೆದ...",CNN's John Sutter sits down with President Oba...
3,3,3,Moscow (CNN)A Russian TV channel aired Hillary...,"Presidential hopeful's video, featuring gay co...",ಸಲಿಂಗಕಾಮಿ ದಂಪತಿಗಳನ್ನು ಒಳಗೊಂಡ ಅಧ್ಯಕ್ಷೀಯ ಭರವಸೆಯ ...,A Russian TV channel aired Hillary Clinton's f...
4,4,4,(CNN)Marco Rubio is all in. The Republican se...,"Raul Reyes: In seeking Latino vote, Marco Rubi...","ರೌಲ್ ರೆಯೆಸ್: ಲ್ಯಾಟಿನೋ ಮತವನ್ನು ಹುಡುಕುವಲ್ಲಿ, ಮಾರ...",Marco Rubio announced he is running for presid...


In [ ]:

print('Original text:',tdf['Document'][8])
print('Actual:',tdf['Summary'][8])
print('Predicted:',tdf['PSummary'][8])


Original text: (CNN)Wanted: film director, must be eager to shoot footage of golden lassos and invisible jets.

CNN confirms that Michelle MacLaren is leaving the upcoming "Wonder Woman" movie (The Hollywood Reporter first broke the story).

MacLaren was announced as director of the movie in November.

CNN obtained a statement from Warner Bros. Pictures that says, "Given creative differences, Warner Bros. and Michelle MacLaren have decided not to move forward with plans to develop and direct 'Wonder Woman' together." (CNN and Warner Bros. Pictures are both owned by Time Warner.)

The movie, starring Gal Gadot in the title role of the Amazon princess, is still set for release on June 23, 2017. It's the first theatrical movie centering around the most popular female superhero.

Gadot will appear beforehand in "Batman v. Superman: Dawn of Justice," due out March 25, 2016.

In the meantime, Warner will need to find someone new for the director's chair.
Actual: Michelle MacLaren is no longe

In [ ]:
tdf=tdf.drop(columns="Document")
tdf=tdf.drop(columns="Unnamed: 0")
tdf=tdf.drop(columns="Unnamed: 0.1")

In [ ]:

tdf.iloc[:,0] = tdf['PSummary'].apply(func=cleanerEng)
tdf.iloc[:,1] = tdf['Kannada_summary'].apply(func= cleanerkan)


In [ ]:
tdf.iloc[:,0] = tdf['Summary'].apply(func= lambda x : (str(x).split()))
tdf.iloc[:,1] = tdf['Kannada_summary'].apply(func= lambda x : (str(x).split()))

In [ ]:
x=prediction(dataset[50][0])
y=' '

print("Actual:",end="")
for idx in dataset[50][1]:
  if idx == 0:
    break
  print(kan_vocab.idx2word[int(idx)],end=' ')
print()
print("Predicted:",y.join(x))

Summary:two years ago the world s youngest country split from its northern nemesis sudan but during a recent visit photographer fabio bucciarelli found ethnic <END> 
Actual:<START> ಎರಡು ವರ್ಷಗಳ ಹಿಂದೆ ವಿಶ್ವದ ಕಿರಿಯ ದೇಶವು ತನ್ನ ಉತ್ತರ ನೆಮೆಸಿಸ್ ಸುಡಾನ್ ನಿಂದ ವಿಭಜನೆಯಾಯಿತು ಆದರೆ ಇತ್ತೀಚಿನ ಭೇಟಿಯ ಸಮಯದಲ್ಲಿ ographer ಾಯಾಗ್ರಾಹಕ ಫ್ಯಾಬಿಯೊ ಬುಕಿಯರೆ <END> 
Predicted: ಎರಡು ವರ್ಷಗಳ ಹಿಂದೆ ವಿಶ್ವದ ಕಿರಿಯ ದೇಶವು ತನ್ನ ಮೂರನೇ ಪ್ರಶಸ್ತಿಯನ್ನು ಪಡೆದರು ಆದರೆ ಇತ್ತೀಚಿನ ಪ್ರಯತ್ನದಲ್ಲಿ ಅಂತಿಮವಾಗಿ ತಿಂಗಳುಗಳಿಂದ ಆದರೆ ತಾಲಿಬಾನ್ ಪರಮಾಣು ಕಾರ್ಯಕ್ರಮದ <END>


In [ ]:
x=['michelle','maclaren','is','leaving','the','wonder','woman','movie','she','was', 'announced','as','director','in','november','the','movie','is','still','set','for','release', 'in', '2017']


In [ ]:
tdf.iloc[:,0] = tdf['Summary'].apply(func= addTokens,start=False)
tdf.iloc[:,1] = tdf['Kannada_summary'].apply(func= addTokens,start=True)

In [ ]:
tdf=tdf.drop(columns="PSummary")

In [ ]:
tdata = tdf.values
test_dataset = vocab(tdata[:,0],token=False)

In [ ]:
res=get(tdf['Summary'][8])
print(res)

Summary:michelle maclaren is leaving the wonder woman movie n she was announced as director in november n the movie is still set for release in 2017 <END> 
['ಕ್ರಿಶ್ಚಿಯನ್', 'ಅಮಾನ್\u200cಪೋರ್', 'ಸಿಎನ್\u200cಎನ್', 'ಐ', 'ರಿಪೋರ್ಟರ್', 'ಬರೆಯುತ್ತಾರೆ', 'ಅದು', 'ಯಾವ', 'ದೊಡ್ಡ', 'ಸಂಬಂಧಿಸಿದೆ', 'ಒಳಗೊಂಡಿದೆ', 'ಅನ್ನು', 'ಸೆಪ್ಟೆಂಬರ್', '7', 'ರಂದು', 'ಕೊನೆಯ', '<END>']


In [ ]:
from tqdm import tqdm
list_refrences = []
list_hypothesis = []
list1=[]
for i in tqdm(range(int(tdata.shape[0]/2))):
  list_refrences.append(tdata[i,0][:-1])
  list1.append(tdata[i,1][1:-1])
  list_hypothesis.append((get(tdata[i,0])[:-1]))

  1%|          | 4/544 [00:00<00:15, 34.86it/s]

Summary:i laughed and learned more from jimmie in one hour than from anyone else says hazzard co star john schneider n james best played bumbling sheriff ros <END> 
Summary:dr anthony the is charged in what authorities say was a failed plot to have another physician hurt or killed n none of anything in this case has <END> 
Summary:cnn s john sutter sits down with president obama for a one on one interview n obama talks about the science behind climate change and public health <END> 
Summary:a russian tv channel aired hillary clinton s first campaign video with a rating stamp that means it s for mature audiences n a clip of the video which <END> 
Summary:marco rubio announced he is running for president n ruben navarrette rubio has been his own worst enemy on what could have been his signature issues <END> 
Summary:the cult sci fi series orphan black returns april 18 n the final season of showtime s nurse jackie begins sunday n the second season of amc s washingt <END> 
Summary:passenge

  2%|▏         | 12/544 [00:00<00:14, 37.41it/s]

Summary:michelle maclaren is leaving the wonder woman movie n she was announced as director in november n the movie is still set for release in 2017 <END> 
Summary:southern monitoring alarm company called police at 12 21 a m april 3 n a grade was applied to the call that meant that no police response was deemed t <END> 
Summary:toya graham tells cnn s anderson cooper 360 she wasn t concerned about embarrassing her son n he was embarrassing himself by wearing that mask and tha <END> 
Summary:in the times the drilling a hole into the skull was thought to be a cure for everything from epilepsy to migraines n but the implement o <END> 
Summary:more than 500 houthi rebels have been killed a saudi official says n a general says the raids aim to keep the rebels from moving toward southern yemen <END> 
Summary:a new cnn orc poll finds 57 feel businesses should be required to serve gay or lesbian couples just as they would heterosexual couples n that s a shif <END> 
Summary:opening statements i

  4%|▎         | 20/544 [00:00<00:13, 37.69it/s]

...

Summary:sally kohn politicians were off guard by backlash over religious freedom bills in arkansas and indiana n kohn companies that sell goods and services t <END> 
Summary:sopranos creator david chase explains the show s final scene n he didn t reveal whether series protagonist tony soprano lives or dies n all i know is <END> 
Summary:to this day i can t believe that i was capable of doing something that terrible arias says n travis alexander s sister says she s trying to block her <END> 
Summary:ruchira gupta 80 of women in mumbai had been street harassed n gupta 1 in 3 women around 

 99%|█████████▊| 536/544 [00:13<00:00, 40.54it/s]

the world face some form of sexual violence at least once in <END> 
Summary:jury selection in the trial of james holmes began in january n holmes faces 165 counts including murder and attempted murder charges n the prosecution <END> 
Summary:the majority of our partners don t have credible and capable special operations forces n it takes years to build special operations forces with credib <END> 
Summary:the delaware family fell ill after renting a villa in the u s virgin islands n the epa says it found a presence of the the in the unit where th <END> 
Summary:christopher the was sentenced to two life terms in prison for killing the and another man at columbia correctional institution in 1994 n he cro <END> 
Summary:the maya the was trapped for 36 hours in the rubble of her the home n the 40 year old mother of four survived without injury apparently pr <END> 
Summary:soviet authorities invaded poland during world war ii and deported hundreds of thousands of poles n some were sent to pr

100%|██████████| 544/544 [00:13<00:00, 39.81it/s]

Summary:google s waze app lets users pinpoint the location of police officers in the field n mark o mara the app poses an enormous risk to deputies and police <END> 
Summary:the boat named the was intercepted some 90 kilometers 56 miles from the libyan port of misrata n an italian naval unit boarded the fishing boat and <END> 
Summary:the the relaunched is an important benchmark in the evolution of the sustainable food movement n it s time that consumer demand for sustainable a <END> 


In [ ]:

import nltk
#print(list1)
#print(list_hypothesis)

bleu1=nltk.translate.bleu_score.corpus_bleu(list1, list_hypothesis, weights=(1.0,0.0,0.0,0.0))
bleu2=nltk.translate.bleu_score.corpus_bleu(list1, list_hypothesis, weights=(0.5,0.5,0.0))
bleu3=nltk.translate.bleu_score.corpus_bleu(list1, list_hypothesis, weights=(0.33,0.33,0.33,0.0))
bleu4=nltk.translate.bleu_score.corpus_bleu(list1, list_hypothesis, weights=(0.25,0.25,0.25,0.25))

print(f"bleu-1 : {nltk.translate.bleu_score.corpus_bleu(list1, list_hypothesis, weights=(1.0,0.0,0.0,0.0))}")
print(f"bleu-2 : {nltk.translate.bleu_score.corpus_bleu(list1, list_hypothesis, weights=(0.5,0.5,0.0))}")
print(f"bleu-3 : {nltk.translate.bleu_score.corpus_bleu(list1, list_hypothesis, weights=(0.33,0.33,0.33,0.0))}")
print(f"bleu-4 : {nltk.translate.bleu_score.corpus_bleu(list1, list_hypothesis, weights=(0.25,0.25,0.25,0.25))}")
bleu_score=(bleu1+bleu2+bleu3+bleu4)/4

[['"ದಿ', 'ಡ್ಯೂಕ್ಸ್', 'ಆಫ್', 'ಹಜಾರ್ಡ್"', 'ನಲ್ಲಿ', 'ಶೆರಿಫ್', 'ಪಾತ್ರವನ್ನು', 'ನಿರ್ವಹಿಸಿದ', 'ಜೇಮ್ಸ್', 'ಬೆಸ್ಟ್', 'ಸೋಮವಾರ', '88', 'ಕ್ಕೆ', 'ನಿಧನರಾದರು', '"ಅಪಾಯ"', '1979', 'ರಿಂದ', '1985', 'ರವರೆಗೆ', 'ನಡೆಯಿತು', 'ಮತ್ತು', 'ಟಿವಿಯಲ'], ['ಡಾ', 'ಆಂಥೋನಿ', 'ಮೊಸ್ಚೆಟ್ಟೊ', 'ಅವರ', 'ವಕೀಲರು', 'ಅವರ', 'ವಿರುದ್ಧದ', 'ಆರೋಪಗಳು', 'ಆಧಾರರಹಿತವಾಗಿವೆ', 'ಎಂದು', 'ಹೇಳುತ್ತಾರೆ', 'ಡ್ರಗ್ಸ್', 'ಮತ್ತು', 'ಶಸ್ತ್ರಾಸ್ತ್ರಗಳನ್ನು', 'ಮಾರಾಟ', 'ಮಾಡಿದ್ದಕ್ಕಾಗಿ', '54', 'ವರ್ಷದ', 'ಮೊ'], ['"ಹವಾಮಾನ', 'ಬದಲಾವಣೆಗಿಂತ', 'ಯಾವುದೇ', 'ಸವಾಲು', 'ಸಾರ್ವಜನಿಕ', 'ಬೆದರಿಕೆಯನ್ನುಂಟುಮಾಡುವುದಿಲ್ಲ"', 'ಎಂದು', 'ಅಧ್ಯಕ್ಷರು', 'ಹೇಳುತ್ತಾರೆ', 'ಕ್ಲೀನ್', 'ಏರ್', 'ಆಕ್ಟ್', 'ಅನ್ನು', 'ಅವರು', 'ಅಮೆರಿಕನ್ನರನ್ನು', '"ಬಹಳಷ್ಟು"'], ['ಸಲಿಂಗಕಾಮಿ', 'ದಂಪತಿಗಳನ್ನು', 'ಒಳಗೊಂಡ', 'ಅಧ್ಯಕ್ಷೀಯ', 'ಭರವಸೆಯ', 'ವೀಡಿಯೊ', 'ರಷ್ಯಾದಲ್ಲಿ', 'ಪ್ರಬುದ್ಧ', 'ರೇಟಿಂಗ್', 'ಪಡೆಯುತ್ತದೆ', 'ರಷ್ಯಾದ', 'ಟಿವಿ', 'ಚಾನೆಲ್', 'ಪ್ರಸಾರ', 'ಮಾಡುವುದರಿಂದ', 'ದೇಶದ', 'ಸಲಿಂಗಕಾಮಿ', 'ವಿರೋ'], ['ರೌಲ್', 'ರೆಯೆಸ್', 'ಲ್ಯಾಟಿನೋ', 'ಮತವನ್ನು', 'ಹುಡುಕುವಲ್ಲಿ', 'ಮಾರ್ಕೊ', 'ರೂಬಿಯೊ', 'ಎರಡು', 'ಪ್ರಮುಖ', 'ವಿಷಯಗಳಲ್ಲಿ', 'ತನ್ನದೇ', 'ಆದ', 'ಕೆಟ್ಟ', 'ಶತ್ರು', 'ವಲಸೆ', 'ಸುಧಾರಣೆ', 'ಕ್ಯೂ

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


bleu-1 : 0.021641311334904653
bleu-2 : 0.14710986144682706
bleu-3 : 0.2822562576496784
bleu-4 : 0.3835490339537137


In [ ]:
print(bleu_score)

0.20863911609628094
